## 5. Planning & Execution
 - simple test scenario: real world single object P&P
* **5.1 Indy**
* **5.2 Panda**
* **5.3 Indy&Panda**

### Prepare the task scene
* Prepare Indy and Panda. Install gripper to indy. (Check release/Figs/5.PlanningAndExecution.jpg)
* Prepare "floor", "track", "goal", "box1" objects as defined in pkg.detector.aruco.marker_config.py
* The robots and "track" are installed on "floor".
* "goal" and "box1" are on "track"
* In this scenario, "box1" will be moved onto "goal" plate

## set running directory to project source

In [1]:
import os
import numpy as np
import time
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

## 5.1 Indy

##### initialize CombinedRobot and GeometryScene
* Prepare the robots as instructed in release/1.Robots
* Connect gripper to indy - match the direction of grippper with the model added below
* Prepare Kinect and Realsense

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *
from pkg.geometry.builder.scene_builder import SceneBuilder

from pkg.controller.combined_robot import CombinedRobot
from pkg.controller.robot_config import RobotConfig, RobotType
INDY_IP = "192.168.0.63"
PANDA_HOST_IP = "192.168.0.172" ## The host computer is the computer that runs control program
PANDA_ROBOT_IP = "192.168.0.13" ## The robot has it's own IP

crob = CombinedRobot(
    robots_on_scene=[
        RobotConfig(0, RobotType.indy7, ((0,-0.3,0), (0,0,0)),
                    INDY_IP),
#         RobotConfig(1, RobotType.panda, ((0,0.3,0), (0,0,0)),
#                     "{}/{}".format(PANDA_HOST_IP, PANDA_ROBOT_IP))
    ], 
    connection_list=[False, False])

from pkg.detector.aruco.stereo import ArucoStereo
from pkg.detector.aruco.marker_config import get_aruco_map
from pkg.detector.camera.kinect import Kinect
from pkg.detector.camera.realsense import RealSense
from pkg.detector.detector_interface import DetectionLevel

stereo = ArucoStereo(aruco_map=get_aruco_map(), 
                     camera_list=[Kinect(), RealSense()])

stereo.initialize()

time.sleep(1) # Let the camera have some time to get stable
stereo.calibrate()
print("Detector initialized")

from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder.instance(detector=stereo)
s_builder.reset_reference_coord(ref_name="floor")

xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob)
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)
ptems = s_builder.add_poles(
    {"cam0": s_builder.ref_coord_inv[:3,3], 
     "cam1":np.matmul(s_builder.ref_coord_inv, stereo.T_c12)[:3,3]},
    color=(0.6,0.6,0.6,0.2))
gscene.show_pose(crob.home_pose)

connection command:
indy0: False
Device configuration: 
	color_format: 3 
	(0:JPG, 1:NV12, 2:YUY2, 3:BGRA32)

	color_resolution: 5 
	(0:OFF, 1:720p, 2:1080p, 3:1440p, 4:1536p, 5:2160p, 6:3072p)

	depth_mode: 3 
	(0:OFF, 1:NFOV_2X2BINNED, 2:NFOV_UNBINNED,3:WFOV_2X2BINNED, 4:WFOV_UNBINNED, 5:Passive IR)

	camera_fps: 2 
	(0:5 FPS, 1:15 FPS, 2:30 FPS)

	synchronized_images_only: False 
	(True of False). Drop images if the color and depth are not synchronized

	depth_delay_off_color_usec: 0 ms. 
	Delay between the color image and the depth image

	wired_sync_mode: 0
	(0:Standalone mode, 1:Master mode, 2:Subordinate mode)

	subordinate_delay_off_master_usec: 0 ms.
	The external synchronization timing.

	disable_streaming_indicator: False 
	(True or False). Streaming indicator automatically turns on when the color or depth camera's are in use.


Start streaming
Detector initialized
Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.


Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran0']/actuator[@name='indy0_motor0']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran1']/actuator[@name='indy0_motor1']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran2']/actuator[@name='indy0_motor2']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran3']/actuator[@name='indy0_motor3']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran4']/actuator[@name='indy0_motor4']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran5']/actuator[@name='indy0_motor5']


Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker


##### add geometries

In [3]:
gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.ENVIRONMENT])
track = gtem_dict["track"]

gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.MOVABLE])
s_builder.give_clearance(track, gtem_dict.values())

##### create PlanningScene

In [4]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

##### create binders

In [5]:
from pkg.geometry.geometry import *
from pkg.planning.constraint.constraint_actor import PlacePlane, Gripper2Tool

gscene.create_safe(GEOTYPE.MESH, "indy0_gripper_vis", link_name="indy0_tcp", dims=(0.1,0.1,0.1),
                   center=(0,0,0), rpy=(0,0,np.pi/2), color=(0.1,0.1,0.1,1), display=True, fixed=True, collision=False, 
                   uri="package://my_mesh/meshes/stl/indy_gripper_asm2_res.STL", scale=(1,1,1))

gscene.create_safe(GEOTYPE.BOX, "indy0_gripper", link_name="indy0_tcp", dims=(0.06,0.08,0.06),
                   center=(0,0,0.04), rpy=(0,0,0), color=(0.0,0.8,0.0,0.5), display=True, fixed=True, collision=True)

gscene.create_safe(GEOTYPE.CYLINDER, "indy0_finger1", link_name="indy0_tcp", dims=(0.03,0.03,0.095),
                   center=(0.006,0.045,0.1), rpy=(0,0,0), color=(0.0,0.8,0.0,0.5), display=True, fixed=True, collision=True)

gscene.create_safe(GEOTYPE.CYLINDER, "indy0_finger2", link_name="indy0_tcp", dims=(0.03,0.03,0.095),
                   center=(-0.006,0.045,0.1), rpy=(0,0,0), color=(0.0,0.8,0.0,0.5), display=True, fixed=True, collision=True)

gscene.create_safe(GEOTYPE.CYLINDER, "indy0_finger3", link_name="indy0_tcp", dims=(0.03,0.03,0.095),
                   center=(0.006,-0.045,0.1), rpy=(0,0,0), color=(0.0,0.8,0.0,0.5), display=True, fixed=True, collision=True)

gscene.create_safe(GEOTYPE.CYLINDER, "indy0_finger4", link_name="indy0_tcp", dims=(0.03,0.03,0.095),
                   center=(-0.006,-0.045,0.1), rpy=(0,0,0), color=(0.0,0.8,0.0,0.5), display=True, fixed=True, collision=True)

gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip0", link_name="indy0_tcp", 
                 dims=(0.01,)*3, center=(0,0,0.14), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)

pscene.create_binder(bname="grip0", gname="grip0", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))
pscene.create_binder(bname="track", gname="track", _type=PlacePlane)
pscene.create_binder(bname="goal", gname="goal", _type=PlacePlane, point=(0,0,0.005))

##### create_subject

In [6]:
from pkg.planning.constraint.constraint_subject import Grasp2Point, PlacePoint
from pkg.planning.constraint.constraint_subject import CustomObject, BoxObject

In [7]:
## create box object with grasping points along positive & negative y-direction and placement point in the bottom face
box_obj = pscene.create_subject(oname="box1", gname="box1", _type=BoxObject)

##### create PlanningPipeline

In [8]:
from pkg.planning.pipeline import PlanningPipeline
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
from pkg.planning.task.rrt import TaskRRT

ppline = PlanningPipeline(pscene)
ppline.set_motion_planner(MoveitPlanner(pscene))
ppline.set_task_planner(TaskRRT(pscene))

##### update state

In [9]:
initial_state = pscene.update_state(crob.home_pose)
print(initial_state.node)

('track',)


##### search for a plan

In [10]:
ppline.search(initial_state, goal_nodes=[("goal",)], verbose=True, 
              display=False, dt_vis=0.01, timeout_loop=100, multiprocess=False, timeout=1, timeout_constrained=10)
schedules = ppline.tplan.find_schedules()
schedules_sorted = ppline.tplan.sort_schedule(schedules)
schedule = ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0])
ppline.play_schedule(schedule, period=0.05)

try: 0 - ('track',)->('grip0',)
try transition motion
transition motion tried: False
result: 0 - ('track',)->('grip0',) = fail
try: 0 - ('track',)->('grip0',)
try transition motion
transition motion tried: False
result: 0 - ('track',)->('grip0',) = fail
try: 0 - ('track',)->('grip0',)
try transition motion
transition motion tried: False
result: 0 - ('track',)->('grip0',) = fail
try: 0 - ('track',)->('grip0',)
try transition motion
transition motion tried: False
result: 0 - ('track',)->('grip0',) = fail
try: 0 - ('track',)->('grip0',)
try transition motion
transition motion tried: False
result: 0 - ('track',)->('grip0',) = fail
try: 0 - ('track',)->('grip0',)
try transition motion
transition motion tried: False
result: 0 - ('track',)->('grip0',) = fail
try: 0 - ('track',)->('grip0',)
try transition motion
transition motion tried: False
result: 0 - ('track',)->('grip0',) = fail
try: 0 - ('track',)->('grip0',)
try transition motion
transition motion tried: False
result: 0 - ('track',)->('

##### execute plan

In [11]:
crob.reset_connection(True)

connection command:
indy0: True
Connect: Server IP (192.168.0.63)


In [12]:
ppline.execute_schedule(schedule, vel_scale=0.5, acc_scale=0.2)

binder: track
rname: None
Connect: Server IP (192.168.0.63)
binder: track
rname: None
Connect: Server IP (192.168.0.63)
go
Connect: Server IP (192.168.0.63)
binder: grip0
rname: indy0
Connect: Server IP (192.168.0.63)
go
Connect: Server IP (192.168.0.63)
binder: goal
rname: None
Connect: Server IP (192.168.0.63)
go
Connect: Server IP (192.168.0.63)
binder: goal
rname: None
Connect: Server IP (192.168.0.63)
Connect: Server IP (192.168.0.63)


## 5.2 Panda

##### initialize CombinedRobot and GeometryScene
* Prepare the robots as instructed in release/1.Robots
* Prepare Kinect and Realsense

In [12]:
from pkg.controller.combined_robot import *
from pkg.project_config import *
from pkg.geometry.builder.scene_builder import SceneBuilder

from pkg.controller.combined_robot import CombinedRobot
from pkg.controller.robot_config import RobotConfig, RobotType
INDY_IP = "192.168.0.63"
PANDA_HOST_IP = "192.168.0.172" ## The host computer is the computer that runs control program
PANDA_ROBOT_IP = "192.168.0.13" ## The robot has it's own IP

crob = CombinedRobot(
    robots_on_scene=[
#         RobotConfig(0, RobotType.indy7, ((0,-0.3,0), (0,0,0)),
#                     INDY_IP),
        RobotConfig(1, RobotType.panda, ((0,0.3,0), (0,0,0)),
                    "{}/{}".format(PANDA_HOST_IP, PANDA_ROBOT_IP))
    ], 
    connection_list=[False, False])

xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob)
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)
ptems = s_builder.add_poles(
    {"cam0": s_builder.ref_coord_inv[:3,3], 
     "cam1":np.matmul(s_builder.ref_coord_inv, stereo.T_c12)[:3,3]},
    color=(0.6,0.6,0.6,0.2))
gscene.show_pose(crob.home_pose)

connection command:
panda1: False
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker


##### add geometries

In [13]:
gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.ENVIRONMENT])
track = gtem_dict["track"]

gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.MOVABLE])
s_builder.give_clearance(track, gtem_dict.values())

##### create PlanningScene

In [14]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

##### create binders

In [15]:
from pkg.geometry.geometry import *
from pkg.planning.constraint.constraint_actor import PlacePlane, Gripper2Tool

gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip1", link_name="panda1_hand", 
                 dims=(0.01,)*3, center=(0,0,0.112), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)

pscene.create_binder(bname="grip1", gname="grip1", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))
pscene.create_binder(bname="track", gname="track", _type=PlacePlane)
pscene.create_binder(bname="goal", gname="goal", _type=PlacePlane, point=(0,0,0.005))

##### create_subject

In [16]:
from pkg.planning.constraint.constraint_subject import Grasp2Point, PlacePoint
from pkg.planning.constraint.constraint_subject import CustomObject, BoxObject

In [17]:
## create box object with grasping points along positive & negative y-direction and placement point in the bottom face
box_obj = pscene.create_subject(oname="box1", gname="box1", _type=BoxObject)

##### create PlanningPipeline

In [18]:
from pkg.planning.pipeline import PlanningPipeline
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
from pkg.planning.task.rrt import TaskRRT

ppline = PlanningPipeline(pscene)
ppline.set_motion_planner(MoveitPlanner(pscene))
ppline.set_task_planner(TaskRRT(pscene))

##### update state

In [19]:
initial_state = pscene.update_state(crob.home_pose)
print(initial_state.node)

('track',)


##### search for a plan

In [20]:
ppline.search(initial_state, goal_nodes=[("goal",)], verbose=True, 
              display=False, dt_vis=0.01, timeout_loop=100, multiprocess=False, timeout=1, timeout_constrained=10)
schedules = ppline.tplan.find_schedules()
schedules_sorted = ppline.tplan.sort_schedule(schedules)
schedule = ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0])
ppline.play_schedule(schedule, period=0.05)

try: 0 - ('track',)->('grip1',)
try transition motion
transition motion tried: False
result: 0 - ('track',)->('grip1',) = fail
try: 0 - ('track',)->('grip1',)
try transition motion
transition motion tried: False
result: 0 - ('track',)->('grip1',) = fail
try: 0 - ('track',)->('grip1',)
try transition motion
transition motion tried: False
result: 0 - ('track',)->('grip1',) = fail
try: 0 - ('track',)->('grip1',)
try transition motion
transition motion tried: True
=============== try reaching goal from ('grip1',) =================
result: 0 - ('track',)->('grip1',) = success
branching: 0->1 (3.14/100.0 s, steps/err: 24(32.8738689423 ms)/0.00217467284189)
try: 1 - ('grip1',)->('goal',)
try transition motion
transition motion tried: False
result: 1 - ('grip1',)->('goal',) = fail
try: 0 - ('track',)->('grip1',)
try transition motion
transition motion tried: False
result: 0 - ('track',)->('grip1',) = fail
try: 1 - ('grip1',)->('goal',)
try transition motion
transition motion tried: False
resul

##### execute plan

In [21]:
crob.reset_connection(True)

connection command:
panda1: True


In [22]:
ppline.execute_schedule(schedule, vel_scale=0.5, acc_scale=0.2)

binder: track
rname: None
binder: track
rname: None
go
binder: grip1
rname: panda1
go
binder: goal
rname: None
go
binder: goal
rname: None


## 5.3 Indy&Panda

##### initialize CombinedRobot and GeometryScene
* Prepare the robots as instructed in release/1.Robots
* Connect gripper to indy - match the direction of grippper with the model added below
* Prepare Kinect and Realsense

In [23]:
from pkg.controller.combined_robot import *
from pkg.project_config import *
from pkg.geometry.builder.scene_builder import SceneBuilder

from pkg.controller.combined_robot import CombinedRobot
from pkg.controller.robot_config import RobotConfig, RobotType
INDY_IP = "192.168.0.63"
PANDA_HOST_IP = "192.168.0.172" ## The host computer is the computer that runs control program
PANDA_ROBOT_IP = "192.168.0.13" ## The robot has it's own IP

crob = CombinedRobot(
    robots_on_scene=[
        RobotConfig(0, RobotType.indy7, ((0,-0.3,0), (0,0,0)),
                    INDY_IP),
        RobotConfig(1, RobotType.panda, ((0,0.3,0), (0,0,0)),
                    "{}/{}".format(PANDA_HOST_IP, PANDA_ROBOT_IP))
    ], 
    connection_list=[False, False])

xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob)
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)
ptems = s_builder.add_poles(
    {"cam0": s_builder.ref_coord_inv[:3,3], 
     "cam1":np.matmul(s_builder.ref_coord_inv, stereo.T_c12)[:3,3]},
    color=(0.6,0.6,0.6,0.2))
gscene.show_pose(crob.home_pose)

connection command:
indy0: False
panda1: False


Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran0']/actuator[@name='indy0_motor0']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran1']/actuator[@name='indy0_motor1']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran2']/actuator[@name='indy0_motor2']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran3']/actuator[@name='indy0_motor3']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran4']/actuator[@name='indy0_motor4']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran5']/actuator[@name='indy0_motor5']


publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker


##### add geometries

In [28]:
gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.ENVIRONMENT])
track = gtem_dict["track"]

gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.MOVABLE])
s_builder.give_clearance(track, gtem_dict.values())

##### create PlanningScene

In [29]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

##### create binders

In [30]:
from pkg.geometry.geometry import *
from pkg.planning.constraint.constraint_actor import PlacePlane, Gripper2Tool

gscene.create_safe(GEOTYPE.MESH, "indy0_gripper_vis", link_name="indy0_tcp", dims=(0.1,0.1,0.1),
                   center=(0,0,0), rpy=(0,0,np.pi/2), color=(0.1,0.1,0.1,1), display=True, fixed=True, collision=False, 
                   uri="package://my_mesh/meshes/stl/indy_gripper_asm2_res.STL", scale=(1,1,1))

gscene.create_safe(GEOTYPE.BOX, "indy0_gripper", link_name="indy0_tcp", dims=(0.06,0.08,0.06),
                   center=(0,0,0.04), rpy=(0,0,0), color=(0.0,0.8,0.0,0.5), display=True, fixed=True, collision=True)

gscene.create_safe(GEOTYPE.CYLINDER, "indy0_finger1", link_name="indy0_tcp", dims=(0.03,0.03,0.095),
                   center=(0.006,0.045,0.1), rpy=(0,0,0), color=(0.0,0.8,0.0,0.5), display=True, fixed=True, collision=True)

gscene.create_safe(GEOTYPE.CYLINDER, "indy0_finger2", link_name="indy0_tcp", dims=(0.03,0.03,0.095),
                   center=(-0.006,0.045,0.1), rpy=(0,0,0), color=(0.0,0.8,0.0,0.5), display=True, fixed=True, collision=True)

gscene.create_safe(GEOTYPE.CYLINDER, "indy0_finger3", link_name="indy0_tcp", dims=(0.03,0.03,0.095),
                   center=(0.006,-0.045,0.1), rpy=(0,0,0), color=(0.0,0.8,0.0,0.5), display=True, fixed=True, collision=True)

gscene.create_safe(GEOTYPE.CYLINDER, "indy0_finger4", link_name="indy0_tcp", dims=(0.03,0.03,0.095),
                   center=(-0.006,-0.045,0.1), rpy=(0,0,0), color=(0.0,0.8,0.0,0.5), display=True, fixed=True, collision=True)

gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip0", link_name="indy0_tcp", 
                 dims=(0.01,)*3, center=(0,0,0.14), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)

gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip1", link_name="panda1_hand", 
                 dims=(0.01,)*3, center=(0,0,0.112), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)

pscene.create_binder(bname="grip0", gname="grip0", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))
pscene.create_binder(bname="grip1", gname="grip1", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))
pscene.create_binder(bname="track", gname="track", _type=PlacePlane)
pscene.create_binder(bname="goal", gname="goal", _type=PlacePlane, point=(0,0,0.005))

##### create_subject

In [31]:
from pkg.planning.constraint.constraint_subject import Grasp2Point, PlacePoint
from pkg.planning.constraint.constraint_subject import CustomObject, BoxObject

In [32]:
## create box object with grasping points along positive & negative y-direction and placement point in the bottom face
box_obj = pscene.create_subject(oname="box1", gname="box1", _type=BoxObject)

##### create PlanningPipeline

In [33]:
from pkg.planning.pipeline import PlanningPipeline
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
from pkg.planning.task.rrt import TaskRRT

ppline = PlanningPipeline(pscene)
ppline.set_motion_planner(MoveitPlanner(pscene))
ppline.set_task_planner(TaskRRT(pscene))

##### update state

In [34]:
initial_state = pscene.update_state(crob.home_pose)
print(initial_state.node)

('track',)


##### search for a plan

In [35]:
ppline.search(initial_state, goal_nodes=[("goal",)], verbose=True, 
              display=False, dt_vis=0.01, timeout_loop=100, multiprocess=False, timeout=1, timeout_constrained=10)
schedules = ppline.tplan.find_schedules()
schedules_sorted = ppline.tplan.sort_schedule(schedules)
schedule = ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0])
ppline.play_schedule(schedule, period=0.05)

try: 0 - ('track',)->('grip0',)
try transition motion
transition motion tried: False
result: 0 - ('track',)->('grip0',) = fail
try: 0 - ('track',)->('grip1',)
try transition motion
transition motion tried: False
result: 0 - ('track',)->('grip1',) = fail
try: 0 - ('track',)->('grip1',)
try transition motion
transition motion tried: True
=============== try reaching goal from ('grip1',) =================
result: 0 - ('track',)->('grip1',) = success
branching: 0->1 (2.07/100.0 s, steps/err: 25(27.0459651947 ms)/0.0014068146692)
try: 1 - ('grip1',)->('goal',)
try transition motion
transition motion tried: False
result: 1 - ('grip1',)->('goal',) = fail
try: 0 - ('track',)->('grip1',)
try transition motion
transition motion tried: True
=============== try reaching goal from ('grip1',) =================
result: 0 - ('track',)->('grip1',) = success
branching: 0->2 (3.12/100.0 s, steps/err: 27(30.2350521088 ms)/0.00119137723403)
try: 2 - ('grip1',)->('goal',)
try transition motion
transition mo

##### execute plan

In [36]:
crob.reset_connection(True, True)

connection command:
indy0: True
panda1: True
Connect: Server IP (192.168.0.63)


In [37]:
ppline.execute_schedule(schedule, vel_scale=0.5, acc_scale=0.2)

binder: track
rname: None
Connect: Server IP (192.168.0.63)
binder: track
rname: None
Connect: Server IP (192.168.0.63)
go
binder: grip1
rname: panda1
Connect: Server IP (192.168.0.63)
go
binder: goal
rname: None
Connect: Server IP (192.168.0.63)
go
binder: goal
rname: None
Connect: Server IP (192.168.0.63)
Connect: Server IP (192.168.0.63)
